# OceanBase Integration with CAMEL

<div class="align-center">
  <a href="https://www.camel-ai.org/"><img src="https://i.postimg.cc/KzQ5rfBC/button.png"width="150"></a>
  <a href="https://discord.camel-ai.org"><img src="https://i.postimg.cc/L4wPdG9N/join-2.png"  width="150"></a></a>
  
⭐ <i>Star us on [*Github*](https://github.com/camel-ai/camel), join our [*Discord*](https://discord.camel-ai.org) or follow our [*X*](https://x.com/camelaiorg)
</div>

This notebook demonstrates how to integrate OceanBase with CAMEL for enhanced data storage and retrieval in multi-agent applications.

In this notebook, you'll explore:

* **OceanBase**: A distributed relational database compatible with MySQL protocol
* **CAMEL Storage**: Using OceanBase as a backend for CAMEL's memory and storage systems
* **Vector Search**: Leveraging OceanBase's vector search capabilities
* **Multi-Agent Applications**: Building robust AI applications with OceanBase persistence

## OceanBase Overview

OceanBase is a distributed relational database developed by Ant Group. Key features include:

- **MySQL Compatibility**: Fully compatible with MySQL protocol
- **High Availability**: Built-in replication and automatic failover
- **Horizontal Scalability**: Scale out by adding more nodes
- **Vector Search**: Support for vector similarity search (OceanBase 4.x)
- **Cost-Effective**: Lower TCO compared to traditional databases

CAMEL can leverage OceanBase for:
- Persistent memory storage
- Vector embeddings storage and retrieval
- Graph storage for agent relationships
- Application data management

## 📦 Installation

First, install CAMEL with OceanBase support:

In [ ]:
# Install CAMEL with OceanBase dependencies
!pip install "camel-ai[all]==0.2.16"  # Latest stable version

# Install OceanBase connector (pyobvector for vector search)
# Note: Requires OceanBase server >= 4.2.0 for vector support
!pip install pyobvector

## 🔑 Setting Up API Keys and OceanBase Connection

In [ ]:
import os

# Set your LLM API key (OpenAI, Anthropic, or others)
os.environ["OPENAI_API_KEY"] = "your-api-key-here"

# OceanBase connection parameters
OCEANBASE_CONFIG = {
    "host": "localhost",          # OceanBase server address
    "port": 2881,                 # OceanBase MySQL port (default: 2881)
    "user": "root",              # Username
    "password": "your-password", # Password
    "database": "camel_db",      # Database name
    "tenant": "sys",             # Tenant name (default: sys)
}

# Export for environment-based configuration
os.environ["OCEANBASE_HOST"] = OCEANBASE_CONFIG["host"]
os.environ["OCEANBASE_PORT"] = str(OCEANBASE_CONFIG["port"])
os.environ["OCEANBASE_USER"] = OCEANBASE_CONFIG["user"]
os.environ["OCEANBASE_PASSWORD"] = OCEANBASE_CONFIG["password"]
os.environ["OCEANBASE_DATABASE"] = OCEANBASE_CONFIG["database"]
os.environ["OCEANBASE_TENANT"] = OCEANBASE_CONFIG["tenant"]

## 🗄️ OceanBase Storage Integration

### Key-Value Storage

In [ ]:
from camel.storages import KeyValueStorage, OceanBaseKVStorage

# Create OceanBase-backed key-value storage
kv_storage = OceanBaseKVStorage(
    config={
        "host": OCEANBASE_CONFIG["host"],
        "port": OCEANBASE_CONFIG["port"],
        "user": OCEANBASE_CONFIG["user"],
        "password": OCEANBASE_CONFIG["password"],
        "database": OCEANBASE_CONFIG["database"],
    },
    table_name="camel_kv_store",
)

# Store data
kv_storage.save("agent_state", {
    "name": "Assistant",
    "role": "helpful_assistant",
    "memory": "User prefers concise responses",
})

# Retrieve data
agent_state = kv_storage.load("agent_state")
print(f"Agent State: {agent_state}")

### Vector Storage for Embeddings

In [ ]:
from camel.storages import VectorStorage, OceanBaseVectorStorage
from camel.embeddings import OpenAIEmbedding

# Initialize embedding function
embedding_func = OpenAIEmbedding()

# Create OceanBase vector storage
vector_storage = OceanBaseVectorStorage(
    config={
        "host": OCEANBASE_CONFIG["host"],
        "port": OCEANBASE_CONFIG["port"],
        "user": OCEANBASE_CONFIG["user"],
        "password": OCEANBASE_CONFIG["password"],
        "database": OCEANBASE_CONFIG["database"],
    },
    embedding_func=embedding_func,
    collection_name="camel_embeddings",
    vector_dim=1536,  # OpenAI text-embedding-ada-002 dimension
)

# Store embeddings
texts = [
    "Machine learning is a subset of artificial intelligence.",
    "Natural language processing enables computers to understand human language.",
    "Vector databases store and retrieve high-dimensional vector representations.",
]

ids = vector_storage.add(texts)
print(f"Stored {len(ids)} embeddings with IDs: {ids}")

### Semantic Search

In [ ]:
# Perform semantic search
query = "What is machine learning?"
results = vector_storage.query(query, top_k=2)

print("Semantic Search Results:")
print("=" * 50)
for i, result in enumerate(results, 1):
    print(f"\n{i}. Text: {result['text'][:100]}...")
    print(f"   Score: {result['score']:.4f}")
    print(f"   ID: {result['id']}")

## 🤖 Multi-Agent with OceanBase Persistence

Build a multi-agent system with persistent memory using OceanBase:

In [ ]:
from camel.agents import ChatAgent
from camel.storages import KeyValueStorage, OceanBaseKVStorage
from camel.memories import Memory, MemoryRecord
from camel.types import RoleType

# Initialize persistent memory storage
memory_storage = OceanBaseKVStorage(
    config={
        "host": OCEANBASE_CONFIG["host"],
        "port": OCEANBASE_CONFIG["port"],
        "user": OCEANBASE_CONFIG["user"],
        "password": OCEANBASE_CONFIG["password"],
        "database": OCEANBASE_CONFIG["database"],
    },
    table_name="camel_agent_memories",
)

# Create memory system
memory = Memory(memory_storage)

# Define agents
assistant_role = "helpful data science tutor"
user_role = "eager student"

# Create agents with persistent memory
assistant = ChatAgent(
    role=assistant_role,
    memory=memory,
)

user = ChatAgent(
    role=user_role,
    memory=memory,
)

### Running a Persistent Conversation

In [ ]:
# Simulate a conversation with memory persistence
conversation_history = []

# User starts a conversation
user_input = "Can you explain what neural networks are?"

# Get assistant response
assistant_response = assistant.step(user_input)

print(f"User: {user_input}")
print(f"Assistant: {assistant_response.msg.content}")

# Store conversation in memory
memory_record = MemoryRecord(
    role=RoleType.ASSISTANT,
    content=assistant_response.msg.content,
    metadata={"user_query": user_input}
)
memory.add(memory_record)

### Retrieving Past Conversations

In [ ]:
# Retrieve past conversations from OceanBase
past_memories = memory.get_all()

print("Past Conversations (Retrieved from OceanBase):")
print("=" * 50)
for i, record in enumerate(past_memories, 1):
    print(f"\n{i}. [{record.role.value}]: {record.content[:100]}...")

# OceanBase ensures this data persists across application restarts
print(f"\n✅ Total memories stored: {len(past_memories)}")
print("📦 Data is persisted in OceanBase and will survive restarts!")

## 🔄 Graph Storage for Agent Relationships

Store agent relationships and interactions in OceanBase:

In [ ]:
from camel.storages import GraphStorage, OceanBaseGraphStorage

# Create OceanBase graph storage
graph_storage = OceanBaseGraphStorage(
    config={
        "host": OCEANBASE_CONFIG["host"],
        "port": OCEANBASE_CONFIG["port"],
        "user": OCEANBASE_CONFIG["user"],
        "password": OCEANBASE_CONFIG["password"],
        "database": OCEANBASE_CONFIG["database"],
    },
)

# Add agent nodes
graph_storage.add_node("assistant_agent", {
    "type": "agent",
    "role": "tutor",
    "created_at": "2024-01-01",
})

graph_storage.add_node("user_agent", {
    "type": "agent",
    "role": "student",
    "created_at": "2024-01-01",
})

# Add relationship edge
graph_storage.add_edge(
    "assistant_agent",
    "user_agent",
    relationship="teaches",
    metadata={"topics": ["machine_learning", "data_science"]}
)

# Query relationships
relationships = graph_storage.get_all_edges()
print("Agent Relationships:")
for edge in relationships:
    print(f"  {edge[0]} --[{edge[2]['relationship']}]--> {edge[1]}")

## 📊 Complete Application Example

A complete multi-agent application with OceanBase persistence:

In [ ]:
"""
Complete Multi-Agent Application with OceanBase Persistence
"""

from camel.agents import ChatAgent
from camel.storages import (
    KeyValueStorage,
    VectorStorage,
    GraphStorage,
    OceanBaseKVStorage,
    OceanBaseVectorStorage,
    OceanBaseGraphStorage,
)
from camel.embeddings import OpenAIEmbedding
from camel.memories import Memory

class OceanBaseCAMELApp:
    """Complete CAMEL application with OceanBase backend."""

    def __init__(self, ob_config):
        self.ob_config = ob_config
        
        # Initialize all storage backends
        self.kv_storage = OceanBaseKVStorage(
            config=ob_config,
            table_name="app_kv_store",
        )
        
        self.vector_storage = OceanBaseVectorStorage(
            config=ob_config,
            embedding_func=OpenAIEmbedding(),
            collection_name="app_embeddings",
            vector_dim=1536,
        )
        
        self.graph_storage = OceanBaseGraphStorage(
            config=ob_config,
        )
        
        self.memory = Memory(self.kv_storage)
        
        # Initialize agents
        self.assistant = ChatAgent(
            role="helpful assistant",
            memory=self.memory,
        )
        
        print("✅ OceanBase CAMEL Application initialized!")

    def chat(self, user_message):
        """Process a user message and return response."""
        # Store user message in vector DB for semantic search
        self.vector_storage.add([user_message])

        # Get assistant response
        response = self.assistant.step(user_message)

        return response.msg.content

    def search_similar_conversations(self, query, top_k=3):
        """Search for similar past conversations."""
        results = self.vector_storage.query(query, top_k=top_k)
        return results

    def get_conversation_graph(self):
        """Get the conversation relationship graph."""
        return self.graph_storage.get_all_edges()

# Initialize the application
app = OceanBaseCAMELApp(OCEANBASE_CONFIG)

## 🏗️ Best Practices

### Connection Pooling
OceanBase supports connection pooling. For production use:

```python
from playhouse.pool import PooledMySQLDatabase

# Create pooled connection
db = PooledMySQLDatabase(
    OCEANBASE_CONFIG["database"],
    max_connections=20,
    timeout=30,
    host=OCEANBASE_CONFIG["host"],
    port=OCEANBASE_CONFIG["port"],
    user=OCEANBASE_CONFIG["user"],
    password=OCEANBASE_CONFIG["password"],
)
```

### Table Creation
OceanBase will automatically create required tables. For production, pre-create tables:

```sql
-- Key-Value Storage Table
CREATE TABLE IF NOT EXISTS camel_kv_store (
    key VARCHAR(255) PRIMARY KEY,
    value LONGTEXT,
    created_at DATETIME DEFAULT CURRENT_TIMESTAMP,
    updated_at DATETIME DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
    INDEX idx_updated_at (updated_at)
) TABLESPACE sysballoon AUTO_INCREMENT=1;

-- Vector Storage Table (requires OceanBase 4.x)
CREATE TABLE IF NOT EXISTS camel_embeddings (
    id BIGINT PRIMARY KEY,
    vector BLOB,
    content LONGTEXT,
    metadata JSON,
    created_at DATETIME DEFAULT CURRENT_TIMESTAMP,
) TABLESPACE sysballoon AUTO_INCREMENT=1;
```

### Error Handling

```python
from peewee import OperationalError

def safe_oceanbase_operation(operation, max_retries=3):
    """Retry OceanBase operations on connection errors."""
    for attempt in range(max_retries):
        try:
            return operation()
        except OperationalError as e:
            if attempt < max_retries - 1:
                print(f"Retry {attempt + 1}/{max_retries}: {e}")
                time.sleep(2 ** attempt)  # Exponential backoff
            else:
                raise
```

## 📈 Performance Considerations

1. **Index Optimization**: Create appropriate indexes on frequently queried columns
2. **Connection Pool Size**: Adjust based on your workload (typically 10-50 connections)
3. **Batch Operations**: Use batch inserts for better performance
4. **Vector Index**: For vector search, create HNSW or IVF index on the vector column
5. **Table Partitioning**: For large tables, consider partitioning by time or other criteria

## 🔗 Additional Resources

- [OceanBase Documentation](https://www.oceanbase.com/docs)
- [CAMEL Documentation](https://docs.camel-ai.org)
- [OceanBase Vector Search](https://www.oceanbase.com/docs/oceanbase-database/oceanbase-database/V4.2.0/vector-search-overview)
- [CAMEL GitHub](https://github.com/camel-ai/camel)

## Summary

This cookbook demonstrated:

✅ Setting up OceanBase connection with CAMEL
✅ Using OceanBase for key-value storage
✅ Implementing vector search with OceanBase
✅ Building multi-agent systems with persistent memory
✅ Managing agent relationships with graph storage
✅ Best practices for production deployment

OceanBase provides a robust, scalable backend for CAMEL multi-agent applications!